<a href="https://colab.research.google.com/github/Spandan123/LargeScaleEnergyFarm/blob/main/perth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/WEC_Perth_49.csv')

print((df))

          X1     Y1      X2      Y2       X3      Y3      X4      Y4     X5  \
0      600.0    0.0  546.16   37.50   489.79   74.88  432.47  112.05  650.0   
1      593.0   12.0  546.16   37.50   489.79   74.88  432.47  112.05  644.0   
2      593.0   12.0  546.16   37.50   489.79   74.88  432.47  112.05  644.0   
3      593.0   12.0  546.16   37.50   489.79   74.88  432.47  112.05  644.0   
4      200.0    0.0  146.17   37.53    89.76   74.93   32.40  112.18  400.0   
...      ...    ...     ...     ...      ...     ...     ...     ...    ...   
36038  100.0  250.0  250.00  250.00   300.00  200.00  400.00  100.00  900.0   
36039  250.0  200.0  500.00  150.00   350.00  200.00  700.00  500.00   50.0   
36040  200.0  100.0  400.00  100.00   350.00  500.00  750.00  200.00  700.0   
36041  500.0  250.0  350.00  350.00  1000.00  250.00  550.00  150.00  800.0   
36042  100.0  400.0  400.00  700.00   400.00  200.00  800.00   50.00  350.0   

          Y5  ...    Power42    Power43    Power44 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Assuming df is already loaded
X = df.iloc[:, :98]  # First 98 columns as features
y = df.iloc[:, -1]   # Last column as target

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=0)

In [ ]:


# Initialize the models
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=0)
gb = GradientBoostingRegressor(n_estimators=100, random_state=0)

# Fit the models on the training data
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
gb.fit(X_train, y_train)

# Predict on the test data
y_pred_lr = lr.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_gb = gb.predict(X_test)

# Calculate the MAPE for each model
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
mape_gb = mean_absolute_percentage_error(y_test, y_pred_gb)

# Print MAPE for each model
print(f"Linear Regression MAPE: {mape_lr * 100:.2f}%")
print(f"Random Forest MAPE: {mape_rf * 100:.2f}%")
print(f"Gradient Boosting MAPE: {mape_gb * 100:.2f}%")

# Ensemble the models using Voting Regressor
ensemble_model = VotingRegressor(estimators=[('lr', lr), ('rf', rf), ('gb', gb)])
ensemble_model.fit(X_train, y_train)

# Predict with the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test)

# Calculate the MAPE for the ensemble model
mape_ensemble = mean_absolute_percentage_error(y_test, y_pred_ensemble)

# Print MAPE for the ensemble model
print(f"Ensemble Model MAPE: {mape_ensemble * 100:.2f}%")


Linear Regression MAPE: 0.94%
Random Forest MAPE: 0.28%
Gradient Boosting MAPE: 0.55%
Ensemble Model MAPE: 0.53%


In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming X and y are your features and target variable
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=0)

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

def mape_loss(y_true, y_pred):
    # Avoid division by zero and compute MAPE
    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true)) * 100

# Define the MLP model with TensorFlow/Keras
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss=mape_loss)

# Train the model using multi-core CPUs (TensorFlow automatically uses multiple cores)
model.fit(X_train, y_train, epochs=200, batch_size=64, verbose=1)

# Evaluate the model on test data
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 96.6484
Epoch 2/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 19.1618
Epoch 3/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9.2681
Epoch 4/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 7.2435
Epoch 5/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.3051
Epoch 6/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.5714
Epoch 7/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.9069
Epoch 8/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.4456
Epoch 9/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.9306
Epoch 10/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.4836
Epoch 11/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.1696
Epoch 12/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.0131
Epoch 13/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.7990
Epoch 14/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.5789
Epoch 15/200
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
y_pred_keras = model.predict(X_test).flatten()
X_stacked = np.column_stack((y_pred_lr, y_pred_rf, y_pred_gb, y_pred_keras))

# Train the final model (XGBoost) on the stacked predictions
final_model = XGBRegressor(n_estimators=100, random_state=0)
final_model.fit(X_stacked, y_test)

# Get predictions from the final model
y_pred_final = final_model.predict(X_stacked)

# Evaluate the performance of the ensemble
mape = mean_absolute_percentage_error(y_test, y_pred_final)
print("Ensemble MAPE:", mape*100)

# Optionally, you can print R2 scores as well
from sklearn.metrics import r2_score
print("Ensemble R2 Score:", r2_score(y_test, y_pred_final))

282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Ensemble MAPE: 0.14114895225748972
Ensemble R2 Score: 0.9896382921912469
